In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow 
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers
from keras import models,layers
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder

In [10]:
def getDataSize(path):
  image_count = {}

  for x in os.listdir(path):
    image_count[x] = len(os.listdir(os.path.join(path, x)))

  return image_count

In [11]:
train_path = 'train'
test_path = 'test'
valid_path = 'valid'

In [12]:
batch_size = 8

In [13]:
def generateImage(path):
  generate_data = ImageDataGenerator(rescale=1.0/255.0,
                                horizontal_flip=True,
                                fill_mode='nearest',
                                zoom_range=0.2,
                                shear_range=0.2,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                rotation_range=30)

  return generate_data.flow_from_directory(path, batch_size=batch_size, target_size=(224, 224), color_mode='grayscale', class_mode='categorical')

In [14]:
def generateImageRGB(path):
  generate_data = ImageDataGenerator(rescale=1.0/255.0,
                                horizontal_flip=True,
                                fill_mode='nearest',
                                zoom_range=0.2,
                                shear_range=0.2,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                rotation_range=30)

  return generate_data.flow_from_directory(path, batch_size=batch_size, target_size=(224, 224), class_mode='categorical')

In [15]:
train_data = generateImage(train_path)
test_data = generateImage(test_path)
valid_data = generateImage(valid_path)

Found 743 images belonging to 4 classes.
Found 127 images belonging to 4 classes.
Found 105 images belonging to 4 classes.


In [16]:
number_of_classes = len(train_data.class_indices)

In [17]:
train_data.class_indices

{'adenocarcinoma': 0,
 'large_cell_carcinoma': 1,
 'normal': 2,
 'squamous_cell_carcinoma': 3}

## VGG

In [42]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.applications import VGG16
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense, Input, Concatenate

# Specify the path to the downloaded VGG16 weights file
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

# Check if the weights file exists
if not os.path.exists(weights_path):
    raise FileNotFoundError(f"File '{weights_path}' not found. Please specify the correct path.")

# Load pre-trained VGG16 model without top layers
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Input layer for grayscale image
input_tensor_gray = Input(shape=(224, 224, 1))

# Convert grayscale to RGB by duplicating the single channel
input_tensor_rgb = Concatenate()([input_tensor_gray, input_tensor_gray, input_tensor_gray])

# Create new model with input_tensor_rgb
new_base_model = Model(input_tensor_gray, base_model(input_tensor_rgb))

# Add custom top layers
x = new_base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(128, activation='relu')(x)
number_of_classes = 4  # Change this according to your number of classes
predictions = Dense(number_of_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=input_tensor_gray, outputs=predictions)

model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 224, 224,  │          0 │ input_layer_14[0… │
│ (Concatenate)       │ 3)                │            │ input_layer_14[0… │
│                     │                   │            │ input_layer_14[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg16 (Functional)  │ (None, 7, 7, 512) │ 14,714,688 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1024)      │    525,312 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 512)       │    524,800 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     65,664 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 4)         │        516 │ dense_6[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,830,980 (60.39 MB)

 Trainable params: 1,116,292 (4.26 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [43]:
import keras

In [44]:
# Compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_data,
    steps_per_epoch=len(train_data),
    epochs=20,
    validation_data=test_data,
    validation_steps=len(test_data)
)


Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


93/93 ━━━━━━━━━━━━━━━━━━━━ 69s 739ms/step - accuracy: 0.3572 - loss: 1.3086 - val_accuracy: 0.5354 - val_loss: 0.8818
Epoch 2/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 192us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/20


2024-04-01 14:40:50.875232: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-04-01 14:40:50.882750: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


93/93 ━━━━━━━━━━━━━━━━━━━━ 76s 819ms/step - accuracy: 0.4930 - loss: 1.0053 - val_accuracy: 0.4567 - val_loss: 0.9958
Epoch 4/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 66us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/20


...

2024-04-01 14:50:33.016844: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-01 14:50:33.018648: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


93/93 ━━━━━━━━━━━━━━━━━━━━ 90s 963ms/step - accuracy: 0.6328 - loss: 0.8257 - val_accuracy: 0.5906 - val_loss: 0.9182
Epoch 18/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 78us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 19/20


2024-04-01 14:52:02.618536: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-01 14:52:02.620545: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


93/93 ━━━━━━━━━━━━━━━━━━━━ 89s 954ms/step - accuracy: 0.6002 - loss: 0.8426 - val_accuracy: 0.5906 - val_loss: 0.8341
Epoch 20/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 82us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00


2024-04-01 14:53:31.373434: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-04-01 14:53:31.375620: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [46]:
# Get the maximum validation accuracy achieved during training
max_val_accuracy = max(history.history['val_accuracy'])

# Print the maximum validation accuracy
print("Best Validation Accuracy:", max_val_accuracy)


Best Validation Accuracy: 0.5905511975288391


In [63]:
from keras.models import load_model

# Save the entire model to a file
model.save('best_model.keras')

# Image Test

In [49]:
test_image = 'test/normal/7.png'

In [54]:
from PIL import Image

# Load a single image for testing
test_image_path = 'test/normal/7.png'
test_img = Image.open(test_image_path)
test_img = test_img.resize((224, 224))  # Resize the image to (224, 224)
test_img_gray = test_img.convert('L')   # Convert the image to grayscale

# Convert the grayscale image to numpy array
test_img_array = np.array(test_img_gray)
test_img_array = np.expand_dims(test_img_array, axis=0)  # Add batch dimension

# Rescale pixel values to [0, 1]
test_img_array = test_img_array / 255.0

# Reshape the array to match the model's input shape
test_img_array = np.expand_dims(test_img_array, axis=-1)  # Add channel dimension

# Now `test_img_array` is ready to be fed to your trained model for prediction


In [58]:
from keras.preprocessing import image
import numpy as np

# Assuming you have already loaded and compiled your model
# model = ...

# Define the target image size
target_size = (224, 224)

# Load and preprocess the single image
test_image_path = 'test/normal/7.png'
test_img = image.load_img(test_image_path, target_size=target_size, color_mode='grayscale')
test_img_array = image.img_to_array(test_img)
test_img_array = np.expand_dims(test_img_array, axis=0)
preprocessed_img = test_img_array / 255.0  # Normalize pixel values

# Now `preprocessed_img` is ready to be fed to your trained model for prediction
output_probabilities = model.predict(preprocessed_img)

# Get the predicted class index
predicted_class_index = np.argmax(output_probabilities)

print("Predicted Class Index:", predicted_class_index)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Predicted Class Index: 2


In [59]:
import sys
import tensorflow as tf
from tensorflow import keras

# Check Python version
python_version = sys.version
print(f"Python version: {python_version}")

# Check TensorFlow version
tensorflow_version = tf.__version__
print(f"TensorFlow version: {tensorflow_version}")



Python version: 3.12.2 (v3.12.2:6abddd9f6a, Feb  6 2024, 17:02:06) [Clang 13.0.0 (clang-1300.0.29.30)]
TensorFlow version: 2.16.1
